In [1]:
pip install --upgrade geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas==1.3.5

  Using cached pandas-1.3.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (12 kB)
Using cached pandas-1.3.5-cp310-cp310-macosx_11_0_arm64.whl (10.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geopandas 1.0.1 requires pandas>=1.4.0, but you have pandas 1.3.5 which is incompatible.
xarray 2023.6.0 requires pandas>=1.4, but you have pandas 1.3.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install rasterio

Note: you may need to restart the kernel to use updated packages.


In [29]:
import os
import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.transform import rowcol
import matplotlib.pyplot as plt

# Define the root directory containing yearly folders
root_directory = './2023/S2A_MSIL2A_20231228T022111_N0510_R003_T52SCG_20231228T054053.SAFE/GRANULE/L2A_T52SCG_A044473_20231228T022111/IMG_DATA/R20m'



# Initialize an empty list to store file paths
jp2_files = []

# Recursively search for .jp2 files
for subdir, _, _ in os.walk(root_directory):
    # Use glob to find all .jp2 files in the current directory
    files = glob.glob(os.path.join(subdir, '*.jp2'))
    # Extend the list with found files
    jp2_files.extend(files)

# Path to your CSV file
csv_file_path = './Address.csv'

# Read the CSV file using pandas
df = pd.read_csv(csv_file_path)

# Assuming your CSV has columns named 'Longitude' and 'Latitude'
# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.Longitude, df.Latitude),
    crs="EPSG:4326"  # Assuming the coordinates are in WGS84
)

# Print the GeoDataFrame
print("GeoDataFrame")
print(gdf)

# If you need to convert the GeoDataFrame to another CRS, for example, EPSG:32652
gdf = gdf.to_crs("EPSG:32652")

# Print the transformed GeoDataFrame
# print("Transformed GeoDataFrame")
# print(gdf)

def extract_values_from_image(image_path, gdf, column_name, scaling_factor=None):
    with rasterio.open(image_path) as dataset:
        print(f"Processing {image_path}")
        print("Image CRS:", dataset.crs)
        print("Image Bounds:", dataset.bounds)
        print("Image Transform:", dataset.transform)

        # Ensure the GeoDataFrame has the same CRS as the image
        if gdf.crs != dataset.crs:
            gdf = gdf.to_crs(dataset.crs)

        # Loop through each point in the GeoDataFrame
        for idx, row in gdf.iterrows():
            # Extract the point coordinates
            point = row.geometry
            x, y = point.x, point.y

            # Get the row and column of the transformed coordinate
            row_idx, col_idx = rowcol(dataset.transform, x, y)

            # Read the value at the row and column
            value = dataset.read(1)[row_idx, col_idx]

            # Add the value to the GeoDataFrame
            gdf.loc[idx, column_name] = value

    return gdf


# Extract values from the images and add them to the GeoDataFrame
gdf = extract_values_from_image(jp2_files[0], gdf, "AOT")
gdf = extract_values_from_image(jp2_files[1], gdf, "Band1")
gdf = extract_values_from_image(jp2_files[2], gdf, "Band2")
gdf = extract_values_from_image(jp2_files[3], gdf, "Band3")
gdf = extract_values_from_image(jp2_files[4], gdf, "Band4")
gdf = extract_values_from_image(jp2_files[5], gdf, "Band5")
gdf = extract_values_from_image(jp2_files[6], gdf, "Band6")
gdf = extract_values_from_image(jp2_files[7], gdf, "Band7")
gdf = extract_values_from_image(jp2_files[8], gdf, "Band11")
gdf = extract_values_from_image(jp2_files[9], gdf, "Band12")
gdf = extract_values_from_image(jp2_files[10], gdf, "B8A(NIR)")
gdf = extract_values_from_image(jp2_files[11], gdf, "SCL")
gdf = extract_values_from_image(jp2_files[12], gdf, "TCI")
gdf = extract_values_from_image(jp2_files[13], gdf, "WVP")

## Print the updated GeoDataFrame
#print("Updated GeoDataFrame with extracted values")

# Save the GeoDataFrame to a CSV file
output_csv_path = './band value/2023/20231228.csv'
gdf.to_csv(output_csv_path, index=False)

# Print the absolute path to confirm where the file was saved
absolute_path = os.path.abspath(output_csv_path)
print(f"CSV file saved at: {absolute_path}")

print(f"GeoDataFrame saved to {output_csv_path}")

GeoDataFrame
                              주소   Latitude   Longitude  \
0                  서울 중구 덕수궁길 15  37.564308  126.974441   
1                서울 용산구 한강대로 405  37.552395  126.971069   
2               서울 종로구 종로35가길 19  37.572051  127.005026   
3                 서울 중구 청계천로 184  37.568617  126.998189   
4                  서울 종로구 종로 169  37.570908  126.996558   
5                서울 용산구 한남대로 136  37.540169  127.004847   
6             서울특별시 광진구 광나루로 571  37.545165  127.095422   
7                서울 성동구 뚝섬로3길 18  37.541996  127.049653   
8                서울 성동구 강변북로 257  37.538879  127.041791   
9                서울 중랑구 용마산로 369  37.584901  127.094035   
10            서울 동대문구 천호대로13길 43  37.575911  127.028966   
11                 서울 동대문구 홍릉로 1  37.580436  127.044497   
12               서울 성북구 삼양로2길 70  37.606676  127.027289   
13            서울 성북구 돈암동 8-164번지  37.603501  127.025965   
14              서울 도봉구 시루봉로2길 34  37.654310  127.029005   
15                서울 은평구 진흥로 215  37.610183

In [ ]:
PRINT(GRP)